# Birdcall noise reduction


I show [this discussion](https://www.kaggle.com/c/birdsong-recognition/discussion/169582#946072).  
I try to remove noise by using Sound Envelope.

version 3:
- I use scipy.ndimage.maximum_filter1d instead of pandas.rolling (ref. [this discussion](https://www.kaggle.com/c/birdsong-recognition/discussion/172921#962037))

In [ ]:
!pip install noisereduce

### import libraly

In [ ]:
import librosa
import random
import IPython

import numpy as np
import pandas as pd
import noisereduce as nr

from pathlib import Path
from matplotlib import pyplot as plt
from scipy.ndimage import maximum_filter1d

### Constants

In [ ]:
INPUT = "../input/birdsong-recognition/"
SEED = 20200807

### Utils

In [ ]:
def audio_to_spec(audio, sr):
    spec = librosa.power_to_db(
        librosa.feature.melspectrogram(audio, sr=sr, fmin=20, fmax=16000, n_mels=128)
    )
    return spec.astype(np.float32)

def envelope(y, rate, threshold):
    mask = []
    y_mean = maximum_filter1d(np.abs(y), mode="constant", size=rate//20)
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask, y_mean

### Denoise

load path

In [ ]:
path_lst = []
for directory in Path(f"{INPUT}/train_audio").iterdir():
    path_lst += [path for path in Path(directory).iterdir()]
    

train_df = pd.read_csv(f"{INPUT}/train.csv")
train_df.head(1)

sampling and load audio data

In [ ]:
random.seed(SEED)
path = random.sample(path_lst, 1)[0]
display(train_df.query(f"filename=='{path.name}'")[["rating", "ebird_code"]])

x, sr = librosa.load(path=path, mono=True)
print("sampling rate:", sr)
plt.plot(x)
plt.show()

IPython.display.Audio(data=x, rate=sr)

I detect point no birdcall by using *Sound Envelope*.


I reffered [this notebook](https://www.kaggle.com/jainarindam/imp-remove-background-dead-noise) and [this discussion](https://www.kaggle.com/c/birdsong-recognition/discussion/169582). (Thank you [Arindam](https://www.kaggle.com/jainarindam)!)

In [ ]:
thr = 0.25
mask, env = envelope(x, sr, thr)

plt.plot(x[mask], label="birdcall")
plt.plot(x[np.logical_not(mask)], label="noise")
plt.legend(bbox_to_anchor=(1, 1), loc='upper right')

noise reduction using spectral gating in python

I show [this discussion](https://www.kaggle.com/c/birdsong-recognition/discussion/169582#946072), and I use [this library](https://pypi.org/project/noisereduce/).  
*audio_clip* is pure audio data and *noise_clip* is low level of sound from Sound Envelope.



In [ ]:
x_denoise = nr.reduce_noise(audio_clip=x, noise_clip=x[np.logical_not(mask)], verbose=True)

In [ ]:
plt.plot(x_denoise)

compare spectrogram

In [ ]:
plt.figure(figsize=(16, 8))
plt.imshow(audio_to_spec(x, sr))
plt.show()

plt.figure(figsize=(16, 8))
plt.imshow(audio_to_spec(x_denoise, sr))
plt.show()

sompare audio

In [ ]:
IPython.display.Audio(data=x, rate=sr)

In [ ]:
IPython.display.Audio(data=x_denoise, rate=sr)

It seems that noise has been removed.

I have made dataset of deonise audio spectrogram image. ([here](https://www.kaggle.com/takamichitoda/birdcall-spectrogram-images/activity))

I train model by use this dataset, I got local fold-0 0.6319398546 and LB 0.543.